<a href="https://colab.research.google.com/github/JayeshK10/Mask-vs-NoMask-Classification/blob/main/Mask_vs_No_Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
import shutil
import random
import pickle


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from numpy import linalg as LA
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import cv2
import matplotlib.image as mpimg
import glob 
import shutil

# Loading Data

In [ ]:
face_data_path = '/content/drive/MyDrive/Data_set/Mini_Proj/AFDB_face_dataset/*/*'
mask_data_path = '/content/drive/MyDrive/Data_set/Mini_Proj/AFDB_masked_face_dataset/*/*'

def image_loader(folder):
  img_path = []
  for img in glob.glob(folder):
    img_path.append(str(img))
  return img_path

no_mask_images = image_loader(face_data_path)
mask_images = image_loader(mask_data_path)

In [ ]:
a = len(mask_images) if (len(no_mask_images) > len(mask_images)) else len(no_mask_images) 
no_mask_img = no_mask_images[0:a]
mask_img = mask_images[0:a]
print('Number of no_mask images' , len(no_mask_img))
print('Number of mask images' , len(mask_img))

Number of no_mask images 2203
Number of mask images 2203


In [ ]:
def feature_extraction (dir_path , label_value):
  data = []
  label = []
  #img_list = []
  for filename in dir_path:
    img = mpimg.imread(filename)
    if img is not None:
      #img_list.append(img)
      gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
      resized = cv2.resize(gray_img,(28, 28), interpolation=cv2.INTER_CUBIC)
      # cv2_imshow(resized)
      data.append(resized.flatten())
      label.append(label_value)
  return data , label

In [ ]:
data_no_mask , label_no_mask = feature_extraction(no_mask_img , 0)
data_mask , label_mask = feature_extraction(mask_img , 1)

# Creating Laten Vector from Images

In [ ]:
X = data_no_mask + data_mask
Y = label_no_mask + label_mask

In [ ]:
if len(X) == len(Y):
  print('Number of data values: ' , len(X))

print('Number of features' , len(X[0]))

Number of data values:  4406
Number of features 784


In [ ]:
X_df = pd.DataFrame(X, columns = list(range(0 , len(X[0]))))
Y_df = pd.DataFrame(Y , columns = ['Label'])

df = pd.concat([X_df, Y_df], axis=1)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,Label
0,46.0,47.0,41.0,41.0,37.0,44.0,38.0,42.0,45.0,39.0,41.0,38.0,40.0,47.0,55.0,61.0,78.0,67.0,68.0,95.0,109.0,80.0,68.0,55.0,43.0,52.0,53.0,49.0,39.0,38.0,32.0,33.0,39.0,40.0,38.0,34.0,33.0,45.0,36.0,26.0,...,220.0,195.0,48.0,33.0,18.0,22.0,39.0,50.0,59.0,59.0,55.0,188.0,177.0,175.0,177.0,177.0,177.0,178.0,182.0,190.0,189.0,190.0,194.0,200.0,199.0,212.0,215.0,209.0,182.0,45.0,8.0,31.0,32.0,26.0,23.0,45.0,62.0,58.0,54.0,0
1,15.0,6.0,15.0,10.0,19.0,18.0,6.0,6.0,9.0,7.0,5.0,6.0,7.0,11.0,7.0,10.0,13.0,14.0,8.0,9.0,1.0,3.0,3.0,3.0,1.0,7.0,8.0,7.0,18.0,14.0,10.0,15.0,14.0,16.0,9.0,4.0,11.0,7.0,8.0,7.0,...,189.0,187.0,187.0,188.0,184.0,179.0,178.0,175.0,168.0,162.0,161.0,26.0,62.0,82.0,90.0,78.0,37.0,38.0,184.0,186.0,194.0,191.0,202.0,200.0,195.0,195.0,193.0,188.0,188.0,183.0,183.0,184.0,181.0,178.0,179.0,174.0,168.0,163.0,154.0,0
2,177.0,136.0,115.0,81.0,68.0,53.0,75.0,77.0,64.0,59.0,47.0,36.0,68.0,66.0,52.0,43.0,36.0,38.0,46.0,37.0,40.0,34.0,45.0,48.0,45.0,39.0,44.0,45.0,156.0,134.0,96.0,62.0,59.0,67.0,104.0,51.0,46.0,47.0,43.0,71.0,...,142.0,139.0,135.0,133.0,124.0,108.0,76.0,108.0,109.0,99.0,88.0,248.0,250.0,247.0,225.0,184.0,213.0,147.0,51.0,51.0,201.0,192.0,187.0,177.0,165.0,156.0,152.0,145.0,142.0,135.0,134.0,122.0,99.0,90.0,114.0,115.0,108.0,107.0,94.0,0
3,106.0,53.0,12.0,9.0,19.0,11.0,10.0,9.0,11.0,11.0,13.0,13.0,13.0,9.0,6.0,18.0,12.0,14.0,11.0,6.0,13.0,11.0,18.0,13.0,24.0,14.0,8.0,16.0,87.0,15.0,15.0,19.0,16.0,14.0,10.0,5.0,5.0,9.0,8.0,12.0,...,98.0,66.0,48.0,32.0,43.0,47.0,53.0,69.0,70.0,81.0,7.0,136.0,112.0,84.0,63.0,14.0,74.0,176.0,221.0,146.0,135.0,158.0,165.0,147.0,126.0,89.0,58.0,47.0,36.0,26.0,48.0,73.0,92.0,112.0,120.0,91.0,75.0,28.0,178.0,0
4,23.0,20.0,27.0,35.0,36.0,34.0,37.0,46.0,52.0,46.0,40.0,44.0,52.0,61.0,87.0,106.0,119.0,126.0,134.0,155.0,162.0,153.0,148.0,108.0,68.0,83.0,108.0,105.0,31.0,47.0,57.0,49.0,37.0,37.0,57.0,65.0,63.0,67.0,73.0,79.0,...,36.0,48.0,70.0,92.0,87.0,93.0,104.0,108.0,102.0,136.0,196.0,96.0,98.0,96.0,94.0,96.0,87.0,78.0,92.0,85.0,86.0,73.0,50.0,71.0,80.0,74.0,82.0,76.0,78.0,91.0,97.0,107.0,98.0,96.0,113.0,87.0,64.0,79.0,113.0,0


# Data preprocessing

shuffle Data

In [ ]:
from sklearn.utils import shuffle
df = shuffle(df)

In [ ]:
col = df.columns
X_Data = df[col[:-1]]
Y_Data = df[col[-1]]

Normal Data

In [ ]:
X_train , X_ , Y_train , Y_ = train_test_split(X_Data, Y_Data, test_size=0.5, random_state=12)
X_test , X_val , Y_test , Y_val = train_test_split(X_, Y_, test_size=0.5, random_state=12)

Standardize Data

In [ ]:
X_scandard = StandardScaler().fit_transform(X_Data)

X_train_std , X_m , Y_train_std , Y_m = train_test_split(X_scandard, Y_Data, test_size=0.5, random_state=12)
X_test_std , X_val_std , Y_test_std , Y_val_std = train_test_split(X_m, Y_m, test_size=0.5, random_state=12)

Dimensionality Reduction

In [ ]:
lda = LDA()
lda_data = lda.fit_transform(X_Data, Y_Data)
X_train_lda, X_m_lda, Y_train_lda, Y_m_lda = train_test_split(lda_data, Y_Data, test_size=0.5, random_state=12)
X_test_lda , X_val_lda , Y_test_lda , Y_val_lda = train_test_split(X_m_lda, Y_m_lda, test_size=0.5, random_state=12)